In [194]:
# Assign 1 Code Skeleton
# Please use this outline to implement your decision tree. You can add any code around this.



In [195]:
import csv

# Enter You Name Here
myname = "Sharanya" 


In [196]:
# Load data set
with open("/content/wine-dataset.csv") as f:
  next(f, None)
  data = [tuple(line) for line in csv.reader(f, delimiter=",")]


print ("Number of records: %d" % len(data))
 

Number of records: 4898


# New Section

In [197]:
import numpy as np
import pandas as pd
import random
from pprint import pprint

In [198]:
df=pd.read_csv("/content/wine-dataset.csv")
df=df.rename(columns={"quality":"label"})

In [199]:
def train_test_split(df,test_ratio):                        #splitting train and test data

  test_size=round(test_ratio*len(df))

  indices=df.index.tolist()
  test_indices=random.sample(population=indices,k=test_size)

  test_df=df.loc[test_indices]
  train_df=df.drop(test_indices)

  return train_df,test_df

In [200]:
#random.seed(0)
train_df,test_df=train_test_split(df,test_ratio=0.1)

In [201]:
data = train_df.values        

In [202]:
def check_purity(data):                 #function to check purity 
  label_column=data[:,-1]
  unique_classes=np.unique(label_column)

  if len(unique_classes)==1:
    return True
  else:
    return False


In [203]:
def classify_data(data):

  label_column=data[:,-1]
  unique_classes, counts_unique_classes=np.unique(label_column, return_counts=True)

  index=counts_unique_classes.argmax()
  classification=unique_classes[index]

  return classification

In [204]:
def get_potential_splits(data):             #calculating all potential splits in the data colunm wise and storing in  a vector
    
    potential_splits = {}
    _, n_columns = data.shape
    for column_index in range(n_columns - 1):        # excluding the last column which is the label
        potential_splits[column_index] = []
        values = data[:, column_index]
        unique_values = np.unique(values)

        for index in range(len(unique_values)):
            if index != 0:
                current_value = unique_values[index]
                previous_value = unique_values[index - 1]
                potential_split = (current_value + previous_value) / 2
                
                potential_splits[column_index].append(potential_split)
    
    return potential_splits

In [205]:
def split_data(data, split_column, split_value):          #function to split data at each decision
    
    split_column_values = data[:, split_column]

    data_below = data[split_column_values <= split_value]
    data_above = data[split_column_values >  split_value]
    
    return data_below, data_above

In [206]:
def calculate_entropy(data):                                  #function to calculate entropy of node in the tree
    
    label_column = data[:, -1]
    _, counts = np.unique(label_column, return_counts=True)

    probabilities = counts / counts.sum()
    entropy = sum(probabilities * -np.log2(probabilities))
     
    return entropy

In [233]:
def calculate_overall_entropy(data_below, data_above):
    
    n = len(data_below) + len(data_above)
    if n==0:
      return 0;
    p_data_below = len(data_below) / n
    p_data_above = len(data_above) / n

    overall_entropy =  (p_data_below * calculate_entropy(data_below) 
                      + p_data_above * calculate_entropy(data_above))
    
    return overall_entropy

In [208]:
def determine_best_split(data, potential_splits):                     #selecting the best split by checking information gain
    
    overall_entropy = 9999
    for column_index in potential_splits:
        for value in potential_splits[column_index]:
            data_below, data_above = split_data(data, split_column=column_index, split_value=value)
            current_overall_entropy = calculate_overall_entropy(data_below, data_above)

            if current_overall_entropy <= overall_entropy:
                overall_entropy = current_overall_entropy
                best_split_column = column_index
                best_split_value = value
    
    return best_split_column, best_split_value



In [209]:
def decision_tree_algorithm(df, counter=0, min_samples=2, max_depth=10):      #recursive decision tree algorithm
    
    # data preparations
    if counter == 0:
        global COLUMN_HEADERS
        COLUMN_HEADERS = df.columns
        data = df.values
    else:
        data = df           
    
    
    # base cases
    if (check_purity(data)) or (len(data) < min_samples) or (counter == max_depth):
        classification = classify_data(data)
        
        return classification

    
    # recursion
    else:    
        counter += 1

        # helper functions 
        potential_splits = get_potential_splits(data)
        split_column, split_value = determine_best_split(data, potential_splits)
        data_below, data_above = split_data(data, split_column, split_value)
        
        # instantiate sub-tree
        feature_name = COLUMN_HEADERS[split_column]
        question = "{} <= {}".format(feature_name, split_value)
        sub_tree = {question: []}
        
        # find answers (recursion)
        yes_answer = decision_tree_algorithm(data_below, counter, min_samples, max_depth)
        no_answer = decision_tree_algorithm(data_above, counter, min_samples, max_depth)
        
        # If the answers are the same, then there is no point in asking the question.
        # This could happen when the data is classified even though it is not pure
        # yet (min_samples or max_depth base cases).
        if yes_answer == no_answer:
            sub_tree = yes_answer
        else:
            sub_tree[question].append(yes_answer)
            sub_tree[question].append(no_answer)
        
        return sub_tree

In [210]:
tree = decision_tree_algorithm(train_df,max_depth=10)
#pprint(tree)

In [211]:
def classify(example, tree):            #classifying the datapoint using tree we constructed
    question = list(tree.keys())[0]
    feature_name, comparison_operator, value = question.split()

    # ask question
    if example[feature_name] <= float(value):
        answer = tree[question][0]
    else:
        answer = tree[question][1]

    # base case
    if not isinstance(answer, dict):
        return answer
    
    # recursive part
    else:
        residual_tree = answer
        return classify(example, residual_tree)

In [212]:
# Accuracy
def calculate_accuracy(df, tree):               #function to calculate overall accuracy

    df["classification"] = df.apply(classify, axis=1, args=(tree,))
    df["classification_correct"] = df["classification"] == df["label"]
    
    accuracy = df["classification_correct"].mean()
    
    return accuracy

In [213]:
# Accuracy
accuracy = calculate_accuracy(test_df, tree)      #printing accuracy
print ("accuracy: %.4f" % accuracy )


accuracy: 0.8184


In [215]:
# Writing results to a file (DO NOT CHANGE)
f = open(myname+"result.txt", "w")
f.write("accuracy: %.4f" % accuracy)
f.close()

In [ ]:
#case for cross validation

In [229]:
def splitDf(df, n) :
    splitPoints = list(map( lambda x: int(x*len(df)/n), (list(range(1,n)))))     
    splits = list(np.split(df.sample(frac=1), splitPoints))
    return splits

def makeTrainAndTest(splits, index) :
   # index is zero based, so range 0-9 for 10 fold split
   test = splits[index]

   leftLst = splits[:index]
   rightLst = splits[index+1:]

   train = pd.concat(leftLst+rightLst)

   return train, test

In [ ]:
df=pd.read_csv("/content/wine-dataset.csv")
df=df.rename(columns={"quality":"label"})
n=10
total_accuracy=0.0;
splits=splitDf(df,n)
for i in range(n):
  train_set,test_set =makeTrainAndTest(splits, i)
  tree = decision_tree_algorithm(train_set,max_depth=10)
  accuracy = calculate_accuracy(test_set, tree) 
  total_accuracy=total_accuracy+accuracy

total_accuracy=total_accuracy/10

print ("accuracy: %.4f" % accuracy )
